In [ ]:
# The aim of this module is to calculate features based on dividend history

# File name convention
# R_ = raw extract 
# E_ = Entry files into enrichment layer
# C_ = enriched layer with calculation
# I = insights layer, designed for model baselines
# V_ = validation


In [8]:
import pandas as pd

DIV = pd.read_csv (r'/Users/joezhou/Downloads/R_ALL_Div.csv',sep='|') 


# DIV12 = DIV.loc[(DIV['Date'] >= pd.to_datetime('now')- pd.DateOffset(months=12))
                    # & (DIV['Date'] < pd.to_datetime('now'))]

#create a baselist for adding on the feature engineered variables
TICKER_LIST = DIV['TickName'].drop_duplicates().reset_index(drop=True)


In [13]:
# Calculate volatility features
DIV_MEDIAN = DIV.groupby("TickName")["Dividends"].median()
DIV_STD = DIV.groupby("TickName")["Dividends"].std()
DIV_MIN = DIV.groupby("TickName")["Dividends"].min()
DIV_MAX = DIV.groupby("TickName")["Dividends"].max()
DIV_SUM = DIV.groupby("TickName")["Dividends"].sum()


TICKER_LIST_ADDED = pd.merge(TICKER_LIST,DIV_MEDIAN,on='TickName').rename(columns={'Dividends': 'Dividends_Median'})
TICKER_LIST_ADDED = pd.merge(TICKER_LIST_ADDED,DIV_STD,on='TickName').rename(columns={'Dividends': 'Dividends_Std'})
TICKER_LIST_ADDED = pd.merge(TICKER_LIST_ADDED,DIV_MIN,on='TickName').rename(columns={'Dividends': 'Dividends_Min'})
TICKER_LIST_ADDED = pd.merge(TICKER_LIST_ADDED,DIV_MAX,on='TickName').rename(columns={'Dividends': 'Dividends_Max'})
TICKER_LIST_ADDED = pd.merge(TICKER_LIST_ADDED,DIV_SUM,on='TickName').rename(columns={'Dividends': 'Dividends_Sum'})

del DIV_MEDIAN, DIV_STD, DIV_MIN, DIV_MAX, DIV_SUM

In [ ]:
#Note: need to loop this later on for 3mth, 6mth, 12mths and 24mths, or some stochastic process

# DIV12_MEDIAN = DIV12.groupby("TickName")["Dividends"].median()
# DIV12_STD = DIV12.groupby("TickName")["Dividends"].std()
# DIV12_MIN = DIV12.groupby("TickName")["Dividends"].min()
# DIV12_MAX = DIV12.groupby("TickName")["Dividends"].max()

# TICKER_LIST_ADDED = pd.merge(TICKER_LIST_ADDED,DIV12_MEDIAN,on='TickName').rename(columns={'Dividends': 'Dividends_Median_12mths'})
# TICKER_LIST_ADDED = pd.merge(TICKER_LIST_ADDED,DIV12_STD,on='TickName').rename(columns={'Dividends': 'Dividends_Std_12mths'})
# TICKER_LIST_ADDED = pd.merge(TICKER_LIST_ADDED,DIV12_MIN,on='TickName').rename(columns={'Dividends': 'Dividends_Min_12mths'})
# TICKER_LIST_ADDED = pd.merge(TICKER_LIST_ADDED,DIV12_MAX,on='TickName').rename(columns={'Dividends': 'Dividends_Max_12mths'})


In [14]:
TICKER_LIST_ADDED.to_csv('/Users/joezhou/Downloads/E_ALL_Div.csv', sep='|', index=False)
